In [10]:
import pandas as pd
import datetime
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff


In [11]:
yandex_russia = pd.read_csv('yandex_russia.csv')

yandex_russia.rename(columns = {'Дата':'Date','Регион':'Region','Заражений':'Confirmed','Выздоровлений':'Recovered',
                               'Смертей':'Deaths','Смертей за день':'Day_deaths',
                               'Заражений за день':'Day_confirmed','Выздоровлений за день':'Day_recovered'},inplace = True)


for i in range(yandex_russia.shape[0]):
    a= yandex_russia.Date[i].split('.')
    yandex_russia.Date[i]=datetime.date(int(a[2]),int(a[1]),int(a[0]))
    
   
yandex_russia.tail(10)

,Date,Region,Confirmed,Recovered,Deaths,Day_deaths,Day_confirmed,Day_recovered
21988,2020-11-25,Ульяновская обл.,26352,21640,315,0,244,315
21989,2020-11-25,Хабаровский край,22905,15284,156,2,318,250
21990,2020-11-25,Хакасия,11157,7399,109,6,182,169
21991,2020-11-25,ХМАО – Югра,33265,28153,381,7,202,185
21992,2020-11-25,Челябинская обл.,23947,21972,244,8,228,65
21993,2020-11-25,Чечня,3854,2429,61,2,91,27
21994,2020-11-25,Чувашия,11940,10817,250,10,119,284
21995,2020-11-25,Чукотский АО,361,294,3,0,6,8
21996,2020-11-25,Ямало-Ненецкий АО,25359,17109,243,3,197,107
21997,2020-11-25,Ярославская обл.,15507,14616,81,3,188,165


In [12]:
df3_temp = yandex_russia.groupby('Date').sum()
df3_temp['Region']='Россия'

y5= pd.concat([yandex_russia,df3_temp.reset_index()],axis = 0)
y5.reset_index()
yandex_russia = y5
yandex_russia.tail(10)

,Date,Region,Confirmed,Recovered,Deaths,Day_deaths,Day_confirmed,Day_recovered
249,2020-11-16,Россия,1948603,1453849,33489,303,22778,13864
250,2020-11-17,Россия,1971013,1475904,33931,442,22410,22055
251,2020-11-18,Россия,1991998,1501083,34387,456,20985,25179
252,2020-11-19,Россия,2015608,1526656,34850,463,23610,25573
253,2020-11-20,Россия,2039926,1551414,35311,461,24318,24758
254,2020-11-21,Россия,2064748,1577435,35778,467,24822,26021
255,2020-11-22,Россия,2089329,1595443,36179,401,24581,18008
256,2020-11-23,Россия,2114502,1611445,36540,361,25173,16002
257,2020-11-24,Россия,2138828,1634671,37031,491,24326,23226
258,2020-11-25,Россия,2162503,1660419,37538,507,23675,25748


In [13]:
# dt = datetime.datetime.today()
# today = datetime.date(dt.year,dt.month,dt.day)
# ystday = datetime.date(today.year,today.month,today.day-1)
# yst2day = datetime.date(today.year,today.month,today.day-2)
# yst3day = datetime.date(today.year,today.month,today.day-3)

# print(today)
# print(ystday)
# print(yst2day)
# print(yst3day)
myday = list(yandex_russia.tail(1).Date)[0]

print(yandex_russia.iloc[-1,0])
print('Выбран день: ',myday)

2020-11-25
Выбран день:  2020-11-25


In [14]:
X = pd.DataFrame()
regions_dict = {}
yandex_russia['Rt'] = 1
yandex_russia['Rt2'] = 1
yandex_russia['Rate_conf'] = 0
yandex_russia['Rate_ill'] = 0

temp_df = yandex_russia.groupby(['Region','Date']).sum()

# Ниже enumerate  для составления словаря из имен регионов {i:j} , для расчетов можно было использовать только j

for i,j in enumerate(temp_df.index.get_level_values(0).unique()):
    temp_df2 =  temp_df.loc[j]
    temp1 = temp_df2['Day_confirmed'].rolling(4).sum()/(temp_df2["Day_confirmed"].rolling(8).sum()-

                                                              temp_df2["Day_confirmed"].rolling(4).sum())
    temp2 = temp_df2['Confirmed'].rolling(4).sum()/(temp_df2['Confirmed'].rolling(8).sum()-

                                                              temp_df2['Confirmed'].rolling(4).sum())
    temp_df2['Rt'] = round(temp1,3)
    temp_df2['Rt2'] = round(temp2,3)
    temp_df2['Region'] = j
    temp_df2['Remaining_ill']=temp_df2['Confirmed']-temp_df2['Recovered']-temp_df2['Deaths']
    temp_df2['MA7_dayconfirmed']=temp_df2['Day_confirmed'].rolling(window=7).mean()
    temp_df2['MA14_dayconfirmed']=temp_df2['Day_confirmed'].rolling(window=14).mean()
    temp_df2['MA7_remaining_ill']=temp_df2['Remaining_ill'].rolling(window=7).mean()
    temp_df2['MA14_remaining_ill']=temp_df2['Remaining_ill'].rolling(window=14).mean()
    
    temp_df2['Change_dayconf'] = 0
    for k in range(1,temp_df2.shape[0]):
        temp_df2['Change_dayconf'].iloc[k]=temp_df2['Day_confirmed'].iloc[k]-temp_df2['Day_confirmed'].iloc[k-1]
    temp_df2['Change_remill'] = 0
    for k in range(1,temp_df2.shape[0]):
        temp_df2['Change_remill'].iloc[k]=temp_df2['Remaining_ill'].iloc[k]-temp_df2['Remaining_ill'].iloc[k-1]
    
    

   #Генератор DF  на каждый регион(сидит в цикле):
   # globals()['x' + str(i)] = temp_df2.reset_index()
    
    X = pd.concat([X,temp_df2.reset_index()],axis = 0)
    regions_dict[i]=j
    
print(regions_dict.items())
    
X = X.reset_index(drop=True)
X.tail()


dict_items([(0, 'Адыгея'), (1, 'Алтай'), (2, 'Алтайский край'), (3, 'Амурская обл.'), (4, 'Архангельская обл.'), (5, 'Астраханская обл.'), (6, 'Башкортостан'), (7, 'Белгородская обл.'), (8, 'Брянская обл.'), (9, 'Бурятия'), (10, 'Владимирская обл.'), (11, 'Волгоградская обл.'), (12, 'Вологодская обл.'), (13, 'Воронежская обл.'), (14, 'Дагестан'), (15, 'Еврейская АО'), (16, 'Забайкальский край'), (17, 'Ивановская обл.'), (18, 'Ингушетия'), (19, 'Иркутская обл.'), (20, 'Кабардино-Балкария'), (21, 'Калининградская обл.'), (22, 'Калмыкия'), (23, 'Калужская обл.'), (24, 'Камчатский край'), (25, 'Карачаево-Черкессия'), (26, 'Карелия'), (27, 'Кемеровская обл.'), (28, 'Кировская обл.'), (29, 'Коми'), (30, 'Костромская обл.'), (31, 'Краснодарский край'), (32, 'Красноярский край'), (33, 'Крым'), (34, 'Курганская обл.'), (35, 'Курская обл.'), (36, 'Ленинградская обл.'), (37, 'Липецкая обл.'), (38, 'Магаданская обл.'), (39, 'Марий Эл'), (40, 'Мордовия'), (41, 'Москва'), (42, 'Московская обл.'), (4

,Date,Confirmed,Recovered,Deaths,Day_deaths,Day_confirmed,Day_recovered,Rt,Rt2,Rate_conf,Rate_ill,Region,Remaining_ill,MA7_dayconfirmed,MA14_dayconfirmed,MA7_remaining_ill,MA14_remaining_ill,Change_dayconf,Change_remill
22252,2020-11-21,14774,14241,73,0,184,321,1.030,1.051,0,0,Ярославская обл.,460,176.285714,170.428571,926.285714,1043.714286,4,-137
22253,2020-11-22,14954,14264,76,3,180,23,1.035,1.051,0,0,Ярославская обл.,614,177.285714,172.071429,846.285714,1005.142857,-4,154
22254,2020-11-23,15137,14289,78,2,183,25,1.043,1.051,0,0,Ярославская обл.,770,178.571429,174.000000,781.714286,969.571429,3,156
22255,2020-11-24,15319,14451,78,0,182,162,1.037,1.050,0,0,Ярославская обл.,790,180.000000,175.857143,718.857143,936.285714,-1,20
22256,2020-11-25,15507,14616,81,3,188,165,1.025,1.050,0,0,Ярославская обл.,810,181.714286,177.714286,696.000000,908.142857,6,20


In [15]:
mask = X.Region == 'Россия'
temp_df = X[mask][['Date','Confirmed','Remaining_ill']]
temp_df.index = temp_df.Date
temp_df.drop(columns =['Date'], inplace = True)
print(temp_df.tail(2))
# print(temp_df.index.get_level_values(0))
# temp_df.loc[datetime.date(2020,6,15),'Россия'].Confirmed
# for i in temp_df.index.get_level_values(0).unique()[-2:]:
#     print(temp_df.loc[i,'Россия'].Confirmed)
rus_date = list(temp_df.index)
rus_conf = list(temp_df.Confirmed)
rus_ill = list(temp_df.Remaining_ill)
rus_dict_conf = dict(zip(rus_date,rus_conf))
rus_dict_ill = dict(zip(rus_date,rus_ill))

print(len(X))
print(X.shape)
print(X.loc[0,'Date'])
for i in range(len(X)):
    X.loc[i,'Rate_conf']=100*X.loc[i,'Confirmed']/rus_dict_conf.get(X.loc[i,'Date'])
    X.loc[i,'Rate_ill']=100*X.loc[i,'Remaining_ill']/rus_dict_ill.get(X.loc[i,'Date'])


            Confirmed  Remaining_ill
Date                                
2020-11-24    2138828         467126
2020-11-25    2162503         464546
22257
(22257, 19)
2020-03-12


In [16]:
mask = (X.Date>=datetime.date(2020,6,13))&(X.Region.isin(['Москва','Россия']))

temp=X[mask]
temp[['Date','Confirmed','Region','Rate_conf','Rate_ill']].tail(20)

,Date,Confirmed,Region,Rate_conf,Rate_ill
14472,2020-11-06,1733440,Россия,100.0,100.0
14473,2020-11-07,1753836,Россия,100.0,100.0
14474,2020-11-08,1774334,Россия,100.0,100.0
14475,2020-11-09,1796132,Россия,100.0,100.0
14476,2020-11-10,1817109,Россия,100.0,100.0
14477,2020-11-11,1836960,Россия,100.0,100.0
14478,2020-11-12,1858568,Россия,100.0,100.0
14479,2020-11-13,1880551,Россия,100.0,100.0
14480,2020-11-14,1903253,Россия,100.0,100.0
14481,2020-11-15,1925825,Россия,100.0,100.0


In [17]:
# russia_regions = pd.read_csv('regions-info.csv')

# russia_regions[(russia_regions.Region =='Московская область')|(russia_regions.Region =='Москва')|
#               (russia_regions.Region =='Санкт-Петербург')].tail()

In [18]:
X.to_csv('X.csv',index=False)
